#  NREL-rex for NSRDB from Eagle and PYSAM simulation

There is an example of calling NSRDB and pysam on https://github.com/NREL/pysam/blob/master/Examples/FetchResourceFileExample.py

The difference is that example downloads the data into a .csv and saves the path file into the `solar_resource_file`.
When having access to Eagle directly, all the data is loaded dynamically from the `.h5` file so saving .csvs slows process, so I want to pass the weather data directly.


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
from rex import NSRDBX
import matplotlib.pyplot as plt

# NSRDBX

In [2]:
#TMY data located on eagle about 900GB
nsrdb_file = '/datasets/NSRDB/current/nsrdb_tmy-2021.h5'

In [3]:
#Input
region = 'Boulder'
region_col = 'county'
parameters = ['air_temperature', 'wind_speed', 'dhi', 'ghi', 'dni', 'surface_albedo']

In [4]:
#Load time and geographical infos
with NSRDBX(nsrdb_file, hsds=False) as f:
    # Get time index
    times = f.time_index
    # Get geographical index for region of interest
    gids = f.region_gids(region=region, region_col=region_col)   
    # Get meta data
    meta = f.meta[f.meta.index.isin(gids)]

In [5]:
#Load weather data
data = []
with NSRDBX(nsrdb_file, hsds=False) as f:
        for p in parameters:
            data.append(f.get_gid_df(p, gids)) #.values

In [6]:
#Create multi-level dataframe
columns = pd.MultiIndex.from_product([parameters, gids], names=["par", "gid"])
df_weather = pd.concat(data, axis=1)
df_weather.columns = columns
df_weather = df_weather.swaplevel(axis=1).sort_index(axis=1)

In [7]:
import pytz

In [8]:
#Create results dataframe
df_res = meta.loc[:, ['latitude', 'longitude']]
df_res['distance'] = np.nan

#loop through dataframe and perform computation 
#at the moment just saving the last location in the county of Boulder for use with pysam
for gid, row in meta.iterrows():
    meta_dict = row.to_dict()
    df_weather_gid = df_weather.loc[:, gid]
    tz_convert_val = meta_dict['timezone']
    df_weather_gid = df_weather_gid.tz_convert(pytz.FixedOffset(tz_convert_val*60))
    #df_weather_gid = df_weather_gid.tz_convert('Etc/GMT+7') # Localizing Data
    # Maping to 2021 because the localizing sets the first values to the year before december...
    df_weather_gid.index =  df_weather_gid.index.map(lambda t: t.replace(year=2021)) 
    # Then rearranging so they are at the end of hte dataframe, becuase I think SAM expect a 8760 starting at Jan 1 0 hours.
    df_weather_gid = df_weather_gid.sort_index()

In [9]:
# Sanity checks
print("MetaData: ", meta_dict)
print("\nWeather DF keys", df_weather_gid.keys())
print("\nDF length", len(df_weather_gid))
df_weather_gid.head(24)

MetaData:  {'latitude': 39.970001220703125, 'longitude': -105.05999755859375, 'elevation': 1614, 'timezone': -7, 'country': 'United States', 'state': 'Colorado', 'county': 'Boulder'}

Weather DF keys Index(['air_temperature', 'dhi', 'dni', 'ghi', 'surface_albedo', 'wind_speed'], dtype='object', name='par')

DF length 8760


par,air_temperature,dhi,dni,ghi,surface_albedo,wind_speed
time_index,,,,,,
2021-01-01 00:30:00-07:00,-13.8,0,0,0,0.8,5.2
2021-01-01 01:30:00-07:00,-13.7,0,0,0,0.8,5.3
2021-01-01 02:30:00-07:00,-13.6,0,0,0,0.8,5.3
2021-01-01 03:30:00-07:00,-13.5,0,0,0,0.8,5.2
2021-01-01 04:30:00-07:00,-13.5,0,0,0,0.8,5.1
2021-01-01 05:30:00-07:00,-13.5,0,0,0,0.8,4.9
2021-01-01 06:30:00-07:00,-13.5,0,0,0,0.8,4.8
2021-01-01 07:30:00-07:00,-12.6,0,0,0,0.8,4.9
2021-01-01 08:30:00-07:00,-10.7,16,0,16,0.8,5.3


In [10]:
df_weather_gid.tail(24)

par,air_temperature,dhi,dni,ghi,surface_albedo,wind_speed
time_index,,,,,,
2021-12-31 00:30:00-07:00,-5.1,0,0,0,0.2,1.6
2021-12-31 01:30:00-07:00,-5.1,0,0,0,0.2,1.2
2021-12-31 02:30:00-07:00,-5.1,0,0,0,0.2,0.5
2021-12-31 03:30:00-07:00,-5.4,0,0,0,0.2,0.4
2021-12-31 04:30:00-07:00,-5.9,0,0,0,0.2,0.7
2021-12-31 05:30:00-07:00,-6.2,0,0,0,0.2,1.2
2021-12-31 06:30:00-07:00,-6.0,0,0,0,0.2,1.9
2021-12-31 07:30:00-07:00,-4.6,0,0,0,0.2,2.5
2021-12-31 08:30:00-07:00,-1.5,32,698,151,0.2,2.6


# bifacialVF

In [11]:
import bifacialvf

In [12]:
bifacialvf.__version__

'0.1.8.1'

In [23]:
meta_dict

{'latitude': 39.970001220703125,
 'longitude': -105.05999755859375,
 'elevation': 1614,
 'timezone': -7,
 'country': 'United States',
 'state': 'Colorado',
 'county': 'Boulder',
 'TZ': -7,
 'Name': 'Boulder',
 'altitude': 1614}

In [14]:
# Variables
tilt = 30                   # PV tilt (deg)
sazm = 180                  # PV Azimuth(deg) or tracker axis direction
cw = 2.0   
albedo = None               # Calculated in previous section from SRRL data. Value is 0.28 up to 11/18/19o
clearance_height=1.5/cw            #1.5m / 2m collector width
gcr = 0.35
pitch = cw/0.4/cw              # 1 / 0.35 where 0.35 is gcr --- row to row spacing in normalized panel lengths. 
rowType = "interior"        # RowType(first interior last single)
transFactor = 0             # TransmissionFactor(open area fraction)
sensorsy = 12                # sensorsy(# hor rows in panel)   <--> THIS ASSUMES LANDSCAPE ORIENTATION 
PVfrontSurface = "glass"    # PVfrontSurface(glass or ARglass)
PVbackSurface = "glass"     # PVbackSurface(glass or ARglass)

# Calculate PV Output Through Various Methods    
calculateBilInterpol = False   # Only works with landscape at the moment.
calculatePVMismatch = False
portraitorlandscape='portrait'   # portrait or landscape
cellsnum = 72
bififactor = 1.0

# Tracking instructions
tracking=False
backtrack=True
limit_angle = 50


In [15]:
savefilevar = 'bifacialvf_results.csv'

In [16]:
df_weather_gid
meta_dict['TZ'] = meta_dict['timezone']
meta_dict['Name'] = meta_dict['county']
meta_dict['altitude'] = meta_dict['elevation']
tilt = np.round(meta_dict['latitude'])
df_weather_gid = df_weather_gid.rename(columns={'dni': 'DNI',
                               'dhi': 'DHI',
                               'ghi': 'GHI'
                              })

In [17]:
bifacialvf.simulate(df_weather_gid, meta_dict, writefiletitle=savefilevar, 
                 tilt=tilt, sazm=sazm, pitch=pitch, clearance_height=clearance_height, 
                 rowType=rowType, transFactor=transFactor, sensorsy=sensorsy, 
                 PVfrontSurface=PVfrontSurface, PVbackSurface=PVbackSurface,
                 albedo=albedo, tracking=tracking, backtrack=backtrack, 
                 limit_angle=limit_angle, calculatePVMismatch=calculatePVMismatch,
                 cellsnum = cellsnum, bififactor=bififactor,
                 calculateBilInterpol=calculateBilInterpol,
                 portraitorlandscape=portraitorlandscape,
                 deltastyle='SAM', agriPV=True)

Calculating Sun position with a delta of +30 mins. i.e. 12 is 12:30 sunpos
No albedo value set or included in TMY3 file (TMY Column name 'Alb (unitless)' expected) Setting albedo default to 0.2
 
 
********* 
Running Simulation for TMY3: 
Location:   Boulder
Lat:  39.970001220703125  Long:  -105.05999755859375  Tz  -7
Parameters: tilt:  40.0   Sazm:  180     Clearance_Height :  0.75   Pitch:  2.5   Row type:  interior   Albedo:  0.2
Saving into bifacialvf_results.csv
 
 
Distance between rows for no shading on Dec 21 at 9 am solar time =  1.8103050439481667
Actual distance between rows =  1.733955556881022
 
Saving Ground Irradiance Values for AgriPV Analysis. 


100%|██████████| 8760/8760 [00:57<00:00, 153.29it/s]

Finished


In [18]:
data, meta = bifacialvf.loadVFresults(savefilevar)
data.set_index(pd.to_datetime(data.date), inplace=True)

In [19]:
ground = data['Ground Irradiance Values'].str.strip('[]').str.split(' ', expand=True).astype(float)

In [20]:
# Calculate geometry
xp = np.cos(np.radians(float(meta['Tilt(deg)'])))
u = int(np.ceil(100*xp/pitch))
b = 100-u
bA = int(np.floor(b/3.0))
bC = int(bA)
bB = int(b-bA-bC)

In [21]:
underpanel = []
bedA = []
bedB = []
bedC = []
for mmonths in range (5, 10):
    datestart = data[data.index.month == mmonths].iloc[0].date
    dateend = data[data.index.month == mmonths].iloc[-1].date
    mask = (data.index >= datestart) & (data.index <= dateend)
    underpanel.append(ground[mask].iloc[:,0:u].mean(axis=1).mean())
    bedA.append(ground[mask].iloc[:,u:u+bA].mean(axis=1).mean())
    bedB.append(ground[mask].iloc[:,u+bA:u+bA+bB].mean(axis=1).mean())
    bedC.append(ground[mask].iloc[:,u+bA+bB:].mean(axis=1).mean())
x = underpanel, bedA, bedB, bedC

In [22]:
x

([165.23882429557005,
  154.9973031891845,
  152.8284417521343,
  207.01741164217006,
  281.1142294850548],
 [193.6939495511164,
  277.10328632139317,
  227.26991709692595,
  103.04215464349238,
  47.573886921790546],
 [433.2518001844046,
  465.9573518648287,
  435.7777967206722,
  389.8337670161424,
  151.10015930465664],
 [422.9151624635386,
  448.0406739140014,
  423.13065178376576,
  436.1937298172972,
  375.0474131531893])

In [ ]:
#x[0][1] #<- undrebed, month June
#x[1][3] #<- bedA, month August